In [1]:
import pandas as pd
import ast
import re
import os

In [6]:
# 1. Function to break open nested dictionary strings (Flattening)
def flatten_cardekho_row(row, city_name):
    flat_data = {'city': city_name}

    def safe_eval(col_name):
        try:
            val = row.get(col_name, '{}')
            # Literal eval handles the string representation of Python dictionaries
            return ast.literal_eval(str(val)) if pd.notna(val) else {}
        except: return {}

    # Extract from 'new_car_detail'
    detail = safe_eval('new_car_detail')
    flat_data.update({
        'fuel_type': detail.get('ft'),
        'body_type': detail.get('bt'),
        'kilometers': detail.get('km'),
        'transmission': detail.get('transmission'),
        'owner_count': detail.get('ownerNo'),
        'brand': detail.get('oem'),
        'model': detail.get('model'),
        'model_year': detail.get('modelYear'),
        'variant': detail.get('variantName'),
        'price': detail.get('price')
    })

    # Extract from 'new_car_overview'
    overview = safe_eval('new_car_overview')
    for item in overview.get('top', []):
        flat_data[f"overview_{item['key'].lower().replace(' ', '_')}"] = item['value']

    # Extract from 'new_car_specs'
    specs = safe_eval('new_car_specs')
    for item in specs.get('top', []):
        flat_data[f"spec_{item['key'].lower().replace(' ', '_')}"] = item['value']

    return flat_data

# 2. Robust numeric cleaning for Price (Handles Lakh, Cr, Crore)
def clean_price(val):
    if pd.isna(val) or val == '': return None
    s = str(val).lower()
    # Find the number part (e.g., '1.30' from '1.30 Crore')
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", s.replace(',', ''))
    if not nums: return None

    num = float(nums[0])
    if 'lakh' in s:
        return num * 100,000
    elif 'cr' in s or 'crore' in s:
        return num * 10,000,000
    return num

# 3. Helper to extract numbers from strings (e.g., '998 CC' -> 998)
def extract_num(val):
    if pd.isna(val) or val == '': return None
    res = re.findall(r"[-+]?\d*\.\d+|\d+", str(val).replace(',', ''))
    return float(res[0]) if res else None

# 4. Main script to loop through all files
def main():
    file_list = [
        'bangalore_cars.xlsx', 'chennai_cars.xlsx', 'delhi_cars.xlsx',
        'kolkata_cars.xlsx', 'hyderabad_cars.xlsx', 'jaipur_cars.xlsx'
    ]

    all_records = []

    for f in file_list:
        if not os.path.exists(f):
            print(f"File not found: {f}")
            continue

        print(f"Processing {f}...")
        # Get city name from the filename
        city = f.split('_')[0].capitalize()

        # Load Excel file
        df = pd.read_excel(f)

        # Flatten each row
        for _, row in df.iterrows():
            all_records.append(flatten_cardekho_row(row, city))

    # Combine into a single Master DataFrame
    master_df = pd.DataFrame(all_records)

    if master_df.empty:
        print("Error: No data found to process.")
        return

    # --- Clean the final table ---
    # Convert '₹ 5 Lakh' to 500000
    master_df['price_numeric'] = master_df['price'].apply(clean_price)

    # Extract numbers from tech specs
    master_df['km_numeric'] = master_df['kilometers'].apply(extract_num)
    master_df['engine_cc'] = master_df.get('spec_engine', pd.Series()).apply(extract_num)
    master_df['mileage_kmpl'] = master_df.get('spec_mileage', pd.Series()).apply(extract_num)

    # Feature engineering: Age of the car
    master_df['car_age'] = 2024 - pd.to_numeric(master_df['model_year'], errors='coerce')

    # Save to final Excel format
    master_df.to_excel('Final_Cleaned_Combined_Cars.xlsx', index=False)
    print("\nSUCCESS! Your combined file is ready: Final_Cleaned_Combined_Cars.xlsx")

if __name__ == "__main__":
    main()

Processing bangalore_cars.xlsx...
Processing chennai_cars.xlsx...
Processing delhi_cars.xlsx...
Processing kolkata_cars.xlsx...
Processing hyderabad_cars.xlsx...
Processing jaipur_cars.xlsx...

SUCCESS! Your combined file is ready: Final_Cleaned_Combined_Cars.xlsx
